see


https://chatgpt.com/c/67b6111a-3698-800e-8564-c1c1aae64c1a

https://weaviate.io/developers/weaviate/quickstart

https://weaviate.io/developers/weaviate/starter-guides/custom-vectors#import-data-and-vectors

In [1]:
!pip install sentence-transformers
!pip install weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [9]:
import json
import requests
from sentence_transformers import SentenceTransformer
from weaviate import Client as WeaviateClient
import weaviate
from weaviate.classes.init import Auth

In [3]:
# Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Weaviate setup
# WEAVIATE_URL = ""  # Replace with your Weaviate URL
# client = WeaviateClient(WEAVIATE_URL)


from google.colab import userdata

# Best practice: store your credentials in environment variables

# weaviate_url = os.environ["WEAVIATE_URL"]
# weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

weaviate_url = userdata.get('WEAVIATE_URL')
weaviate_api_key = userdata.get("WEAVIATE_API_KEY")


# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())


True


In [11]:
def process_and_import_paragraphs(paragraphs, collection_name="Paragraph"):
    """
    Processes a list of paragraphs, generates vectors, creates a JSON file, and imports into Weaviate.
    """

    try:
        # Check if collection exists; if not, create it
        if collection_name not in client.collections.list()["collections"]:
            schema = {
                "classes": [
                    {
                        "class": collection_name,
                        "properties": [
                            {"name": "text", "dataType": ["text"]},
                        ],
                        "vectorizer": "none"  # We're providing vectors
                    }
                ]
            }
            client.schema.create(schema)
    except Exception as e:
        print(f"Error creating schema: {e}")
        return


    data_objects = []
    json_data = [] # for the JSON file
    for paragraph in paragraphs:
        vector = model.encode(paragraph).tolist()  # Encode paragraph to vector
        data_object = {
            "text": paragraph,
            "vector": vector,
        }
        json_data.append(data_object)

        weaviate_object = client.data.DataObject(
            properties={
                "text": paragraph,
            },
            vector=vector
        )

        data_objects.append(weaviate_object)


    # Save to JSON file
    with open("paragraphs_with_vectors.json", "w") as f:
        json.dump(json_data, f, indent=4)


    # Import data into Weaviate
    try:
        collection = client.collections.get(collection_name)
        collection.data.insert_many(data_objects)
        print(f"Successfully imported {len(data_objects)} paragraphs into Weaviate.")
    except Exception as e:
        print(f"Error importing data into Weaviate: {e}")




In [13]:
# Example usage:
paragraphs = [
    "Weaviate is an open-source vector database designed for scale.",
    "Sentence transformers help convert text into vector representations.",
    "This is a simple example for importing data into Weaviate."
]

# Convert paragraphs to vectors
vectors = model.encode(paragraphs)



In [14]:
data_to_import = []
for i, (para, vector) in enumerate(zip(paragraphs, vectors)):
    data_to_import.append({
        "id": f"para-{i}",
        "properties": {
            "paragraph": para
        },
        "vector": vector.tolist()  # Convert numpy array to list for JSON serialization
    })


In [15]:
json_filename = "paragraphs_with_vectors.json"
with open(json_filename, 'w') as json_file:
    json.dump(data_to_import, json_file, indent=4)

print(f"Data saved to {json_filename}")


Data saved to paragraphs_with_vectors.json


In [ ]:
# create collection


questions = client.collections.create(
    name="Question",
    vectorizer_config=Configure.Vectorizer.text2vec_cohere(),   # Configure the Cohere embedding integration
    generative_config=Configure.Generative.cohere()             # Configure the Cohere generative AI integration
)